In [1]:

import os
import cv2
from os.path import join,abspath
from natsort import natsorted
import numpy as np
from glob import glob

In [2]:
from path import *

In [3]:
for key,value in paths.items():
    paths[key]=join('..',value)

In [4]:
paths

{'Data': '../Data',
 'original': '../Data/original',
 'custom': '../Data/custom',
 'neg_custom': '../Data/custom/n',
 'custom_p': '../Data/custom/p',
 'custom_p_train': '../Data/custom/p/train',
 'custom_p_test': '../Data/Test/custom',
 'sample': '../Data/custom/sample',
 'public': '../Data/Public',
 'neg_public': '../Data/Public/Negative',
 'celeba': '../Data/Public/celeba',
 'public_p': '../Data/Public/p',
 'public_p_train': '../Data/Public/p/train',
 'public_p_test': '../Data/Test/public',
 'res': '../Result',
 'res_cus': '../Result/Custom',
 'res_pub': '../Result/Public',
 'src': '../src',
 'utils': '../utils',
 'turtorial': '../turtorial'}

In [5]:
np.random.seed(101)

In [6]:
haar_cascade = cv2.CascadeClassifier('../haarcascade_frontalface_alt.xml')

Creating cropped custom positive samples

In [7]:
custom_dire=glob(paths['original']+'/**/*.jpg',recursive=True)+glob(paths['sample']+'/**/*.jpg',recursive=True)

In [8]:
## augumentation 
import albumentations as A

In [9]:
transform=A.Compose([A.HorizontalFlip(0.5),
A.Rotate(limit=35),
A.RandomBrightnessContrast(p=0.6),
A.ZoomBlur(max_factor=1.15,p=0.25)])


In [10]:
##shuffling the custom postive samples
np.random.shuffle(custom_dire)

In [12]:

for num,img_dire in enumerate(custom_dire):
    img=cv2.imread(img_dire)
    if num<=np.math.ceil(len(custom_dire)*0.8):
        
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces_rect = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9)
        for i,(x,y,w,h) in enumerate(faces_rect):
            crop=img[y:y+h,x:x+w,:]
            crop=cv2.resize(crop,(96,96))
            crop=cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
            
            new_dire=paths['custom_p_train']
            if not os.path.exists(new_dire):
                os.makedirs(new_dire)
            new_img_path=join(new_dire,'c_{}_{}.jpg'.format(num,i))
            cv2.imwrite(new_img_path,crop)
    else:
        new_dire=paths['custom_p_test']
        if not os.path.exists(new_dire):
            os.makedirs(new_dire)

        new_img_path=join(new_dire,'{}_{}.jpg'.format(num,i))
        cv2.imwrite(new_img_path,img)
        
    
        
        
        


In [23]:
## Augument and Crop
for num,img_dire in enumerate(custom_dire):
    img=cv2.imread(img_dire)
    transformed=transform(image=img)
    img=transformed["image"]
    if num<=np.math.ceil(len(custom_dire)*0.8):

        
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces_rect = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9)
        for i,(x,y,w,h) in enumerate(faces_rect):
            crop=img[y:y+h,x:x+w,:]
            crop=cv2.resize(crop,(96,96))
            crop=cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
            
            
            new_dire=paths['custom_p_train']
            if not os.path.exists(new_dire):
                os.makedirs(new_dire)
            new_img_path=join(new_dire,'c_{}_aug_{}.jpg'.format(num,i))
            cv2.imwrite(new_img_path,crop)
   
        
    
        

In [15]:
## Creating Negative Custom Samples 
dires=glob(join(paths['custom'],'neg_video')+'/*.mp4')

In [18]:
for i,dire in enumerate(dires):
    cap=cv2.VideoCapture(dire)
    j=0
    while cap.isOpened():
        sucess,frame=cap.read()
        if not sucess:
            break
        if j%10!=0:
            j+=1
            continue
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame=cv2.resize(frame,(96,96))
        transformed=transform(image=frame)
        aug_1=transformed["image"]
        aug_2=transformed["image"]
        targe_folder=paths['neg_custom']
        target_dire=join(targe_folder,'{:02d}_{:04d}.jpg'.format(i,j))
        target_aug1_dire=join(targe_folder,'{:02d}_aug1_{:04d}.jpg'.format(i,j))
        target_aug2_dire=join(targe_folder,'{:02d}_aug2_{:04d}.jpg'.format(i,j))
        cv2.imwrite(target_dire,frame)
        cv2.imwrite(target_aug1_dire,aug_1)
        cv2.imwrite(target_aug2_dire,aug_2)
        j+=1


Creating cropped Public Dataset

In [8]:


img_dires=glob(paths['celeba']+'/*')

img_dires=img_dires[:10000]

for num,img_dire in enumerate(natsorted(img_dires)):
    img=cv2.imread(img_dire)
    if num<=np.math.ceil(len(img_dires)*0.8):
    
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces_rect = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9)
        for i,(x,y,w,h) in enumerate(faces_rect):
            crop=img[y:y+h,x:x+w,:]
            crop=cv2.resize(crop,(96,96))
            crop=cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        
            new_dire=paths['public_p_train']
            if not os.path.exists(new_dire):
                os.makedirs(new_dire)
            new_img_path=join(new_dire,'{}_{}.jpg'.format(num,i))
            cv2.imwrite(new_img_path,crop)



    else:
        new_dire=paths['public_p_test']
    
        if not os.path.exists(new_dire):
            os.makedirs(new_dire)
        new_img_path=join(new_dire,'{}_{}.jpg'.format(num,i))
        cv2.imwrite(new_img_path,img)

Added more Public Dataset

In [26]:

img_dires=glob(paths['celeba']+'/*')

img_dires=img_dires[11000:16000]

for num,img_dire in enumerate(natsorted(img_dires)):
    img=cv2.imread(img_dire)
    if num<=np.math.ceil(len(img_dires)*0.8):
    
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces_rect = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9)
        for i,(x,y,w,h) in enumerate(faces_rect):
            crop=img[y:y+h,x:x+w,:]
            crop=cv2.resize(crop,(96,96))
            crop=cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        
            new_dire=paths['public_p_train']
            if not os.path.exists(new_dire):
                os.makedirs(new_dire)
            new_img_path=join(new_dire,'n_{}_{}.jpg'.format(num,i))
            cv2.imwrite(new_img_path,crop)



    else:
        new_dire=paths['public_p_test']
    
        if not os.path.exists(new_dire):
            os.makedirs(new_dire)
        new_img_path=join(new_dire,'n_{}_{}.jpg'.format(num,i))
        cv2.imwrite(new_img_path,img)


In [17]:
crop.shape

(96, 96, 3)

Negative public samples


In [5]:
neg_src_dire='/home/akashghimire/Downloads/Negative/'

In [6]:
neg_dires=glob(neg_src_dire+'**/*.png',recursive=True)+glob(neg_src_dire+'**/*.jpg',recursive=True)
np.random.shuffle(neg_dires)
neg_dires=neg_dires[:60000]

In [7]:

for i,img_dire in enumerate(neg_dires):
    img=cv2.imread(img_dire)
    if img is None:
        
        continue
    if img.shape[2]==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_rect = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9)
    if len(faces_rect)>0:
        continue

    else:
        img=cv2.resize(gray_img,(96,96))
        new_dire=join('..',paths['neg_public'],'{:07d}.jpg'.format(i))
        print(cv2.imwrite(new_dire,img))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [37]:
paths

{'Data': '../Data',
 'original': '../Data/original',
 'custom': '../Data/custom',
 'neg_custom': '../Data/custom/n',
 'custom_p': '../Data/custom/p',
 'custom_p_train': '../Data/custom/p/train',
 'custom_p_test': '../Data/Test/custom',
 'sample': '../Data/custom/sample',
 'public': '../Data/Public',
 'neg_public': '../Data/Public/Negative',
 'celeba': '../Data/Public/celeba',
 'public_p': '../Data/Public/p',
 'public_p_train': '../Data/Public/p/train',
 'public_p_test': '../Data/Test/public',
 'src': '../src',
 'utils': '../utils',
 'turtorial': '../turtorial'}

Adding more public Negative samples

In [12]:
neg_src_dire='/media/akashghimire/28E0-A271/Negative/'
neg_dires=glob(neg_src_dire+'**/*.png',recursive=True)+glob(neg_src_dire+'**/*.jpg',recursive=True)
np.random.shuffle(neg_dires)


for i,img_dire in enumerate(neg_dires):
    img=cv2.imread(img_dire)
    if img is None:
        
        continue
    if img.shape[2]==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_rect = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=9)
    if len(faces_rect)>0:
        continue

   
    img=cv2.resize(gray_img,(96,96))
    new_folder=join(paths['public'],'n')
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)
    new_dire=join(new_folder,'n_{}.jpg'.format(i))
    print(cv2.imwrite(new_dire,img))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [10]:
paths['public']

'../Data/Public'

In [36]:
len(neg_dires)

5892

In [32]:
os.path.exists(new_dire)

False

In [33]:
cv2.imwrite(new_dire,img)

False

In [37]:
gray_img.shape


(512, 512)

In [18]:
len(os.listdir(join('..',paths['neg_public'])))

32766